# Creation of Bioschemas property pages

Based on the JSON-LD from DDE, corresponding to Bioschemas released types, we create a page for each Bioschemas property declared in that file. This script needs to be re-run and pages committed and merged whenever there is a change in the releases types. It uses the modified Bioschemas types file that already uses the canonical Bioschemas URL https://bioschemas.org/terms/

## Installation of packages

In [1]:
import sys
!{sys.executable} -m pip install rdflib

In [2]:
import sys
!{sys.executable} -m pip install pyyaml

In [3]:
from rdflib import Graph
from rdflib import URIRef

from rdflib.namespace import RDFS

## Load the Bioschemas released types JSON-LD as RDF

In [6]:
#load graph
g = Graph()
g.parse("https://bioschemas.org/types/bioschemas_types.jsonld")

schema_domain = URIRef("https://schema.org/domainIncludes")
schema_range = URIRef("https://schema.org/rangeIncludes")
bs_ns = "https://bioschemas.org/terms/"

## For each property, get the domain and name, and create the corresponding page

In [11]:
for s in g.subjects(predicate=schema_domain, unique=True) :
    prop_name = str(s).split('/')[-1]
    
    if bs_ns in s : #it is a bs property, we want to create a page for it
        prop_yaml_str = 'prop_name: ' + prop_name + '\n'
        prop_yaml_str +=  'redirect_from: ' + "/terms/" + prop_name + '\n'
        prop_yaml_str +=  'description: "' + g.value(s, RDFS.comment) + '"\n'

        all_domain = 'used_in_types: \n'
        for prop_domain in g.objects(subject=s, predicate=schema_domain) :
            dom_name = str(prop_domain).split('/')[-1]
            all_domain += '- ' + dom_name + '\n'

        all_range = 'expected_values_as_type: \n' 
        for prop_range in g.objects(subject=s, predicate=schema_range) :
            range_name = str(prop_range).split('/')[-1]
            all_range += '- ["' + range_name + '", "' + str(prop_range) + '"]\n'

        prop_yaml_str += all_domain
        prop_yaml_str += all_range
        with open('../pages/_properties/' + prop_name + '.html', 'w') as file:
            file.write("---\n")
            file.write(prop_yaml_str)
            file.write("\n---\n\n")
            file.write("{% include properties.html %}\n")

### ToDo - Improvements 
- Move the code to a script that can be integrated into GitHub actions
- Do something similar for draft properties (requires a modification of the landing page for properties as right now is only a list, supporting released and draft types would require tabs similar to types page)
